## GICS 체계로 클러스터링

In [2]:
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.metrics import davies_bouldin_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from sklearn.inspection import permutation_importance
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
import matplotlib.font_manager as fm
import platform
import shap
import lime
import lime.lime_tabular
from sklearn.manifold import TSNE
import random  


# CSV 파일 경로
gics_file_path = '../csv/종목_GICS분류_클러스터.csv'
kospi_file_path = '../csv/financial_data_processing_cospi200.csv'


In [3]:
df_processing_cospi200 = pd.read_csv(kospi_file_path)
df_gics = pd.read_csv(gics_file_path)


In [4]:
df_processing_cospi200

,기업명_그룹,매출액,영업이익,당기순이익,자산총계,자본총계,자본금,영업활동현금흐름,이자발생부채,영업이익률,순이익률,ROE(%),ROA(%),부채비율,EPS(원),PER(배),BPS(원),PBR(배),발행주식수(보통주)
0,BGF리테일,78168.10,2353.90,1806.20,30413.75,10074.50,173.00,6499.40,3371.05,3.0045,2.3025,19.1640,6.1845,204.4665,10452.15,13.842000,58318.90,2.5360,17283906.0
1,BNK금융지주,82983.75,9329.10,7427.55,1402323.10,106667.95,16297.00,1059.20,194600.65,12.1160,9.5230,7.2460,0.5505,1212.4140,2177.75,3.688000,31087.35,0.2545,323323988.3
2,CJ,399896.75,21261.65,4553.45,459940.30,173529.85,1790.00,43187.50,182317.25,5.2905,1.1735,3.2850,1.0215,164.8735,4743.30,23.256000,157376.10,0.5830,29176998.0
3,CJ대한통운,117828.20,4457.35,2185.80,95146.65,40917.65,1141.00,6244.35,33233.10,3.7705,1.8440,5.1870,2.3095,132.5530,8222.50,18.852000,183922.25,0.6120,22812344.0
4,CJ제일제당,284474.55,14863.60,6214.55,290387.75,115612.75,819.00,20663.90,110318.45,5.2365,2.2195,6.7945,2.2100,151.2490,25676.20,16.468500,433280.45,0.7875,15054186.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,현대해상,168437.95,10420.80,7558.30,476717.70,60701.60,447.00,14037.90,16655.60,6.3260,4.5935,12.1730,1.5880,746.7925,8454.75,3.715000,77333.60,0.3630,89400000.0
191,호텔신라,39464.80,361.90,-312.55,31713.35,7943.10,2000.00,1265.10,16232.85,0.8090,-0.8505,-3.0005,-0.9120,336.9375,-781.90,3.286643,20980.10,3.7785,39248121.0
192,효성중공업,40082.75,2242.65,1152.10,49684.20,13957.55,466.00,2791.35,12929.00,5.2690,2.6040,8.2110,2.2965,269.5100,11180.00,21.237749,135929.25,1.2290,9324548.0
193,효성티앤씨,77966.55,3994.95,2516.70,42602.50,15773.25,216.00,5586.25,13711.75,5.0295,3.1635,18.9360,6.5965,177.7275,45108.25,33.786000,295216.55,1.1600,4327682.0


### Davies-Bouldin Index 계산

In [5]:
# 절대경로로 불러옴
df_2020_2022 = pd.read_csv("../csv/financial_data_window_2020_2022.csv")
df_2021_2023 = pd.read_csv("../csv/financial_data_window_2021_2023.csv")
df_2022_2024 = pd.read_csv("../csv/financial_data_window_2022_2024.csv")

# 기업명을 따로 저장한 후, feature만 따로 분리함
company_2020_2022 = df_2020_2022['기업명_그룹']
company_2021_2023 = df_2021_2023['기업명_그룹']
company_2022_2024 = df_2022_2024['기업명_그룹']
X_raw_2020_2022 = df_2020_2022.drop(columns=["기업명_그룹"])
X_raw_2021_2023 = df_2021_2023.drop(columns=["기업명_그룹"])
X_raw_2022_2024 = df_2022_2024.drop(columns=["기업명_그룹"])

X_raw_2020_2022

,매출액,영업이익,당기순이익,자산총계,자본총계,자본금,영업활동현금흐름,이자발생부채,영업이익률,순이익률,ROE(%),ROA(%),부채비율,EPS(원),PER(배),BPS(원),PBR(배),발행주식수(보통주)
0,70985.166667,2197.000000,1664.000000,2.733767e+04,8672.000000,173.0,5791.166667,7519.166667,3.071667,2.326667,20.556667,6.370000,216.680000,9629.666667,18.258333,50201.166667,3.486667,17283906.0
1,65153.666667,10667.500000,7884.333333,1.302548e+06,104070.666667,16297.0,13934.833333,188141.666667,16.550000,12.281667,8.033333,0.630000,1150.475000,2278.500000,3.096667,29234.333333,0.236667,325935246.0
2,372903.000000,19360.833333,6464.000000,4.477637e+05,169206.666667,1790.0,36457.000000,181936.166667,5.175000,1.726667,4.171667,1.501667,164.290000,5808.000000,17.345000,150724.833333,0.565000,29176998.0
3,116434.333333,3747.500000,1749.333333,9.451867e+04,40292.666667,1141.0,5215.333333,32803.333333,3.213333,1.496667,3.723333,1.875000,134.561667,5678.333333,28.503333,176992.666667,0.658333,22812344.0
4,278437.666667,15670.833333,8298.833333,2.822718e+05,110611.666667,819.0,17094.166667,106516.166667,5.633333,3.008333,10.466667,3.048333,154.990000,37635.166667,10.200000,392853.166667,0.983333,15054186.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,169082.333333,11569.000000,8470.166667,4.893132e+05,72241.666667,447.0,9939.333333,11728.833333,7.235000,5.293333,13.046667,1.703333,665.428333,9474.166667,3.616667,91999.166667,0.303333,89400000.0
187,42520.500000,478.666667,-632.833333,2.837017e+04,5662.000000,2000.0,1157.333333,16159.000000,0.873333,-1.751667,-9.115000,-2.041667,402.983333,-1582.166667,3.356680,14950.833333,5.455000,39248121.0
188,32839.500000,1189.833333,368.333333,4.304900e+04,10586.666667,466.0,677.500000,14398.833333,3.580000,1.130000,2.238333,0.911667,305.755000,2210.333333,34.394582,101515.833333,0.678333,9324548.0
189,81669.500000,5808.000000,3736.166667,4.139467e+04,14648.500000,216.0,4552.666667,14565.000000,7.075000,4.560000,30.030000,10.281667,193.871667,65943.333333,67.425000,274582.666667,1.400000,4327682.0


## GICS Window Separation

In [6]:
# 3. VIF 계산 및 제거
def remove_multicollinearity(df, threshold=10.0):
    while True:
        X_const = sm.add_constant(df)
        vif = pd.Series(
            [variance_inflation_factor(X_const.values, i) for i in range(X_const.shape[1])],
            index=X_const.columns
        )
        vif = vif.drop("const")
        max_vif = vif.max()
        if max_vif > threshold:
            drop_col = vif.idxmax()
            df = df.drop(columns=[drop_col])
        else:
            break
    return df

### 2020~2022

In [7]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
from sklearn.metrics import davies_bouldin_score

# 1. GICS 병합
df_gics_renamed = df_gics.rename(columns={'종목명': '기업명_그룹'})
df_merged_2020_2022 = pd.merge(df_2020_2022, df_gics_renamed[['기업명_그룹', 'Cluster']], on='기업명_그룹')

# 2. feature, label 분리
labels_2020_2022 = df_merged_2020_2022['Cluster']
features_raw = df_merged_2020_2022.drop(columns=['기업명_그룹', 'Cluster'])



features_filtered = remove_multicollinearity(features_raw)

# 4. 정규화 (스케일링)
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features_filtered)

# 5. DBI 계산
dbi_2020_2022 = davies_bouldin_score(features_scaled, labels_2020_2022)
print("✅ 2020–2022 GICS DBI (VIF 제거 후):", dbi_2020_2022)


✅ 2020–2022 GICS DBI (VIF 제거 후): 5.017694477014077


### 2021~2023

In [8]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
from sklearn.metrics import davies_bouldin_score

# 1. GICS 병합
df_gics_renamed = df_gics.rename(columns={'종목명': '기업명_그룹'})
df_merged_2021_2023 = pd.merge(df_2021_2023, df_gics_renamed[['기업명_그룹', 'Cluster']], on='기업명_그룹')

# 2. feature, label 분리
labels_2021_2023 = df_merged_2021_2023['Cluster']
features_raw = df_merged_2021_2023.drop(columns=['기업명_그룹', 'Cluster'])



features_filtered = remove_multicollinearity(features_raw)

# 4. 정규화 (스케일링)
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features_filtered)

# 5. DBI 계산
dbi_2021_2023 = davies_bouldin_score(features_scaled, labels_2021_2023)
print("✅ 2020–2022 GICS DBI (VIF 제거 후):", dbi_2021_2023)


✅ 2020–2022 GICS DBI (VIF 제거 후): 5.152391854221761


### 2022~2024

In [9]:

# 1. GICS 병합
df_gics_renamed = df_gics.rename(columns={'종목명': '기업명_그룹'})
df_merged_2022_2024 = pd.merge(df_2022_2024, df_gics_renamed[['기업명_그룹', 'Cluster']], on='기업명_그룹')

# 2. feature, label 분리
labels_2022_2024 = df_merged_2022_2024['Cluster']
features_raw = df_merged_2022_2024.drop(columns=['기업명_그룹', 'Cluster'])



features_filtered = remove_multicollinearity(features_raw)

# 4. 정규화 (스케일링)
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features_filtered)

# 5. DBI 계산
dbi_2022_2024 = davies_bouldin_score(features_scaled, labels_2022_2024)
print("✅ 2022–2024 GICS DBI (VIF 제거 후):", dbi_2022_2024)


✅ 2022–2024 GICS DBI (VIF 제거 후): 4.729771922920844
